In [ ]:
from asteroid.data import LibriMix
from torch.utils.data import DataLoader

# LibriMix 데이터셋 로드
train_set = LibriMix(
    root='./data/LibriMix',  # 다운로드할 경로
    task='sep_clean',        # clean 또는 noisy 버전 선택
    sample_rate=16000,       # 샘플링 속도
    n_src=2,                 # 화자의 수 (Libri2Mix or Libri3Mix)
    subset='train'           # 학습 데이터셋 선택
)

# 검증 데이터셋 로드
valid_set = LibriMix(
    root='./data/LibriMix',
    task='sep_clean',
    sample_rate=16000,
    n_src=2,
    subset='valid'           # 검증 데이터셋 선택
)

# DataLoader로 학습 데이터 및 검증 데이터 설정
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=4)

In [ ]:
from asteroid.models import ConvTasNet
import torch

# 모델 정의 (Conv-TasNet 사용)
model = ConvTasNet(n_src=2)  # 두 명의 화자 분리

# 손실 함수 및 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()  # 손실 함수로 MSE 사용

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for mix, sources in train_loader:
        # 데이터 준비
        mix, sources = mix.to(device), sources.to(device)

        # 모델에 입력하고 예측값 생성
        optimizer.zero_grad()
        output = model(mix)

        # 손실 계산 및 역전파
        loss = criterion(output, sources)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def validate(model, valid_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for mix, sources in valid_loader:
            mix, sources = mix.to(device), sources.to(device)

            # 검증 단계에서 예측값 생성
            output = model(mix)

            # 손실 계산
            loss = criterion(output, sources)
            total_loss += loss.item()

    return total_loss / len(valid_loader)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    valid_loss = validate(model, valid_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")

In [ ]:
# LibriMix 테스트 데이터셋 로드
test_set = LibriMix(
    root='./data/LibriMix',
    task='sep_clean',  # 깨끗한 데이터 사용 (or noisy for noisy version)
    sample_rate=16000,
    n_src=2,  # 화자의 수
    subset='test'  # 테스트 데이터셋 선택
)

test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

In [ ]:
import soundfile as sf
import numpy as np
from asteroid.metrics import get_metrics
import os

# 테스트 함수
def test_model(model, test_loader, device, output_dir='./separated_audio'):
    model.eval()

    # 오디오 파일을 저장할 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    with torch.no_grad():
        for idx, (mix, sources) in enumerate(test_loader):
            mix, sources = mix.to(device), sources.to(device)

            # 모델을 사용해 예측
            output = model(mix)

            # 성능 평가 (SDR 등 계산)
            mix_np = mix.squeeze(0).cpu().numpy()  # (batch_size, seq_len)
            sources_np = sources.squeeze(0).cpu().numpy()  # (n_src, seq_len)
            output_np = output.squeeze(0).cpu().numpy()  # (n_src, seq_len)

            # SDR 등 평가 지표 계산
            metrics = get_metrics(mix_np, sources_np, output_np, sample_rate=16000)
            print(f"Sample {idx+1}, SDR: {metrics['sdr']}, SIR: {metrics['sir']}, SAR: {metrics['sar']}")

            # 분리된 오디오 파일 저장
            for i, est_source in enumerate(output_np):
                output_file = os.path.join(output_dir, f"sample{idx+1}_speaker{i+1}.wav")
                sf.write(output_file, est_source, 16000)  # 16kHz로 저장
                print(f"Saved {output_file}")

    print("Testing completed.")

In [ ]:
# 훈련된 모델을 테스트
test_model(model, test_loader, device)